In [39]:
import numpy as np
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
import random as rand
import powerlaw

In [2]:
#early visualization - Olivia
    #bipartite graph in networkx
    #upload node and edge-list to gephi
#some stats - Olivia
    #networkx
#genre based collaboration
    #require data-scraping on artist genre
#influential song-writers
    #require data-scraping on original artist
#review lecture topics and incorporate analysis where appropriate
#put together into slide deck

In [2]:
#upload original csv
cover_df = pd.read_csv('/Users/oliviasato/Desktop/Network Science/covers_with_artists.csv')
#get rid of nans
cover_df = cover_df.fillna('')

#put song titles in same rows as artists
count = 0
for i in range(0,len(cover_df)):
    if cover_df['Song'][i] == '':
        cover_df['Song'][i] = cover_df['Song'][i-1]
    if cover_df['Song'][i][0] == '-':
        cover_df['Track ID'][i] = ''

columns = ['Song', 'Track ID', 'Artist ID', 'Artist Name']
no_empties = pd.DataFrame(columns=columns)
for i in range(0,len(cover_df)):
    if not cover_df['Track ID'][i] == '':
        row_to_add = pd.DataFrame(cover_df.iloc[i]).T
        no_empties = pd.concat([no_empties, row_to_add], ignore_index=True)
cover_df = no_empties

In [3]:
#add og artist and year
df = pd.read_csv('/Users/oliviasato/Desktop/Network Science/og_artist_data.csv')
df = df.fillna('')
for i in range(0,len(df)):
    if not df['Release Year'][i] == '':
        df['Release Year'][i] = int(df['Release Year'][i])
og_art_year = df.rename(columns={'SongID': 'Song ID'})

/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_21626/2775737992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release Year'][i] = int(df['Release Year'][i])


In [4]:
#only run after uploading csv, otherwise splitting gets weird
#probs fix later, just need to work with copies instead of og
#make 2 node lists, one of unique songs and another of unique artists
artists = []
songs = []
for i in range(len(cover_df)):  
    if not (cover_df['Artist Name'][i] in artists):
        artists.append(cover_df['Artist Name'][i])
    if not (cover_df['Song'][i] in songs):
        songs.append(cover_df['Song'][i])

#seperating song IDs and adding to seperate column
song_id = []
songs_cleaned = []
for i in range(0,len(cover_df)):
    txt = cover_df['Song'][i]
    split = re.split(', ', txt)
    songid = split[0]
    title = split[-1]
    song_id.append(songid)
    songs_cleaned.append(title)
songs = songs_cleaned

for i in range(0,len(cover_df)):
    cover_df['Song'][i] = songs[i]

cover_df['Song ID'] = song_id
cover_df['Original Artist'] = len(cover_df)*[()]
cover_df['Original Year'] = len(cover_df)*[()]
#eventually add year of cover here
cover_df = cover_df[['Song ID', 'Song', 'Original Artist', 'Original Year', 'Track ID', 'Artist ID', 'Artist Name']]

cover_df

,Song ID,Song,Original Artist,Original Year,Track ID,Artist ID,Artist Name
0,"72636,4253",My Sweet Lord,(),(),TRPYNNL12903CAF506,ARXJJSN1187B98CB37,Bebe Winans
1,"72636,4253",My Sweet Lord,(),(),TRFYRVZ128F92EF998,ARNUFGE1187B9B7881,THE CHIFFONS
2,"72636,4253",My Sweet Lord,(),(),TRXAJXI128F4267A92,AR7K9W71187B9AF065,Jane Olivor
3,"72636,4253",My Sweet Lord,(),(),TRMONTS128F427FF78,ARHYUI71187FB48366,Billy Preston
4,"72636,4253",My Sweet Lord,(),(),TRGSXCN128F9320D4B,ARZEAO01187B998042,Jody Miller
...,...,...,...,...,...,...,...
16144,5225,Night Train,(),(),TRADNTU128F932B04A,ARRX4FO1187FB58DD6,Jackie Davis
16145,5225,Night Train,(),(),TRYYPZE128F93129FB,ARRDLVE1187FB48F11,James Brown
16146,5225,Night Train,(),(),TRGTCNC128F4269C48,ARY123V1187B9A5BDF,Paul Revere & The Raiders (featuring Mark Lind...
16147,61856,At Last I Am Free,(),(),TRVZNAL128F427F1E5,ARUEMHA1187B9B875E,Elizabeth Fraser


In [5]:
#take only second song id when there are two
id2 = []
for i in range(0,len(cover_df)):
    txt = cover_df['Song ID'][i]
    split = re.split(',', txt)
    second = split[-1]
    id2.append(second)

for i in range(0,len(cover_df)):
    cover_df['Song ID'][i] = id2[i]
cover_df

/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_21626/3132458665.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cover_df['Song ID'][i] = id2[i]


,Song ID,Song,Original Artist,Original Year,Track ID,Artist ID,Artist Name
0,4253,My Sweet Lord,(),(),TRPYNNL12903CAF506,ARXJJSN1187B98CB37,Bebe Winans
1,4253,My Sweet Lord,(),(),TRFYRVZ128F92EF998,ARNUFGE1187B9B7881,THE CHIFFONS
2,4253,My Sweet Lord,(),(),TRXAJXI128F4267A92,AR7K9W71187B9AF065,Jane Olivor
3,4253,My Sweet Lord,(),(),TRMONTS128F427FF78,ARHYUI71187FB48366,Billy Preston
4,4253,My Sweet Lord,(),(),TRGSXCN128F9320D4B,ARZEAO01187B998042,Jody Miller
...,...,...,...,...,...,...,...
16144,5225,Night Train,(),(),TRADNTU128F932B04A,ARRX4FO1187FB58DD6,Jackie Davis
16145,5225,Night Train,(),(),TRYYPZE128F93129FB,ARRDLVE1187FB48F11,James Brown
16146,5225,Night Train,(),(),TRGTCNC128F4269C48,ARY123V1187B9A5BDF,Paul Revere & The Raiders (featuring Mark Lind...
16147,61856,At Last I Am Free,(),(),TRVZNAL128F427F1E5,ARUEMHA1187B9B875E,Elizabeth Fraser


In [6]:
#add original artist and release year according to matching Song ID
og_artist = []
og_year = []

for i in range(0,len(cover_df)):
    try:
        songid = cover_df['Song ID'][i]
        artist = og_art_year.loc[og_art_year['Song ID'] == int(songid), 'Original Artist'].iloc[0]
        year = og_art_year.loc[og_art_year['Song ID'] == int(songid), 'Release Year'].iloc[0]
    except IndexError:
        artist = ''
        year = 0
        # if not temp.empty:
        #     artist = temp['Original Artist']
        #     year = temp['Release Year']
    og_artist.append(artist)
    og_year.append(year)

for i in range(0,len(cover_df)):
    cover_df['Original Artist'][i] = og_artist[i]
    cover_df['Original Year'][i] = og_year[i]
    
cover_df

/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_21626/518839011.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cover_df['Original Artist'][i] = og_artist[i]
/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_21626/518839011.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cover_df['Original Year'][i] = og_year[i]


,Song ID,Song,Original Artist,Original Year,Track ID,Artist ID,Artist Name
0,4253,My Sweet Lord,The Chiffons [US1],1962,TRPYNNL12903CAF506,ARXJJSN1187B98CB37,Bebe Winans
1,4253,My Sweet Lord,The Chiffons [US1],1962,TRFYRVZ128F92EF998,ARNUFGE1187B9B7881,THE CHIFFONS
2,4253,My Sweet Lord,The Chiffons [US1],1962,TRXAJXI128F4267A92,AR7K9W71187B9AF065,Jane Olivor
3,4253,My Sweet Lord,The Chiffons [US1],1962,TRMONTS128F427FF78,ARHYUI71187FB48366,Billy Preston
4,4253,My Sweet Lord,The Chiffons [US1],1962,TRGSXCN128F9320D4B,ARZEAO01187B998042,Jody Miller
...,...,...,...,...,...,...,...
16144,5225,Night Train,,0,TRADNTU128F932B04A,ARRX4FO1187FB58DD6,Jackie Davis
16145,5225,Night Train,,0,TRYYPZE128F93129FB,ARRDLVE1187FB48F11,James Brown
16146,5225,Night Train,,0,TRGTCNC128F4269C48,ARY123V1187B9A5BDF,Paul Revere & The Raiders (featuring Mark Lind...
16147,61856,At Last I Am Free,,0,TRVZNAL128F427F1E5,ARUEMHA1187B9B875E,Elizabeth Fraser


In [28]:
fixed_names = cover_df
for i in range(0,len(fixed_names)):
    artist = fixed_names['Artist Name'][i]
    if '&' in artist:
        parsed = re.split('&', artist)
        fixed = parsed[0]+ 'and' + parsed[1]
        fixed_names['Artist Name'][i] = fixed
cover_df = fixed_names

artists = []
songs = []
for i in range(len(cover_df)):  
    if not (cover_df['Artist Name'][i] in artists):
        artists.append(cover_df['Artist Name'][i])
    if not (cover_df['Song'][i] in songs):
        songs.append(cover_df['Song'][i])

/var/folders/dc/rvt7flcd7qzcbq_87lq3c8rr0000gn/T/ipykernel_21626/2318849414.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_names['Artist Name'][i] = fixed


In [29]:
#make network
from networkx.algorithms import bipartite
G = nx.Graph()

#add nodes
G.add_nodes_from(songs, bipartite=0)
G.add_nodes_from(artists, bipartite=1)

#add links
for i in range(0,len(cover_df)):
    song = cover_df['Song'][i]
    artist = cover_df['Artist Name'][i]
    G.add_edge(song,artist)

In [17]:
final_csv = 'data_011123.csv'
cover_df.to_csv(final_csv, index=False)

In [32]:
graph_dict = {
    "nodes": [{"id": node} for node in G.nodes()],
    "links": [{"source": source, "target": target} for source, target in G.edges()]
}

# Save the graph as GML
output_file = "data_011123_1.gml"
nx.write_gml(G, output_file)

In [62]:
artist_projection = bipartite.projected_graph(G, artists)
S = [artist_projection.subgraph(c).copy() for c in nx.connected_components(artist_projection)]
largest_cc = S[0]

degree = largest_cc.degree()
degree = dict(degree)
# degree = list(degree.values())

total_degrees = sum(degree.values())
average_degree = total_degrees / len(largest_cc)

print('avg deg = ' + str(average_degree))

clust = nx.transitivity(largest_cc)
len(largest_cc.nodes)

avg deg = 12.362557427258805


5224

In [55]:
nx.average_shortest_path_length(largest_cc)

4.0371736772709

In [58]:
largest_cc.average_degree()

AttributeError: 'Graph' object has no attribute 'average_degree'

In [35]:
def plot_degree(degree, number_of_bins, log_binning, base): 
    if log_binning:
        log = np.log2 if base == 2 else np.log10
    lower_limit = log(min(degree))
    upper_limit = log(max(degree))
    bins = np.logspace(lower_limit,upper_limit,number_of_bins,base = base) 
    y, __ = np.histogram(degree, bins = bins, density = True)
    x = bins[1:] - np.diff(bins)/2.0 
    return x,y

In [64]:
#Quick check of implementation
degree = largest_cc.degree()
degree = dict(degree)
degree = list(degree.values())
x1, y1 = plot_degree(degree, 10, True, 10)

results = powerlaw.Fit(degree)
alpha = results.power_law.alpha
print(alpha)
xmin = results.power_law.xmin
print(xmin)

# powerlaw.plot_pdf(degree)

# #Compare to scale-free
# def link_select_model(n):
#     G = nx.Graph()
#     nodes = [0,1]
#     edges = [(0,1)]
#     G.add_nodes_from(nodes)
#     G.add_edge(0,1)
#     for i in range(2,n):
#         #pick random link first (avoid new self-loop)
#         #range because third node can only pick one edge
#         chosen_link = edges[rand.randint(0,i-2)]
#         #pick random node on link
#         chosen_node = chosen_link[rand.randint(0,1)]
#         #add node to graph
#         G.add_node(i)
#         #add edge to graph
#         G.add_edge(i,chosen_node)
#         edges = list(G.edges())
#         #Add checkpoints for 3 orders of magnitude
#     return G

# link_compare = link_select_model(5225)
# degree = link_compare.degree()
# degree = dict(degree)
# degree = list(degree.values())
# x2, y2 = plot_degree(degree, 10, True, 10)

# fig, ax = plt.subplots()
# ax.scatter(x1,y1, c = 'blue')
# # ax.plot(
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_xlabel('k',fontsize=14)
# ax.set_ylabel('p \u2096',fontsize=14)
# ax.set_title('Artist Projection Degree Distibution', fontsize=15)

Calculating best minimal value for power law fit
3.772209137970111%
70.0


In [49]:
max(degree)

250

In [165]:
average_degree = sum(dict(largest_cc.degree()).values()) / len(largest_cc)
total_degree = sum(dict(largest_cc.degree()).values())
prob_link = average_degree/total_degree
prob_link

0.00019138755980861245

In [26]:
cover_df['Song'].nunique()

4836

In [ ]:
'''
Network:
bipartite
undirected

Links:
unweighted
undirected
time attribute - year of cover

Song Nodes:
original artist
streams (optional)
year of original release
themes - lyrical content (optional)

Artist Nodes:
genre
significance - streams or number of songs
'''